In [58]:
import pandas as pd
from datetime import datetime, date, timedelta
import numpy as np
import altair as alt

In [26]:
# This was a tester from the blog post to figure out how to calculate beta.

x0 = np.array([35240, 45071, 55674, 69660, 86570, 105363, 125415])
x1 = np.array([45071, 55674, 69660, 86570, 105363, 125415, 143444])
active_cases = np.array([34100, 43374, 53468, 66488, 82439, 100022, 118522])
cum_rec = np.array([668, 1096, 1400, 2107, 2807, 3612, 4685])
cum_dead = np.array([472, 601, 806, 1065, 1324, 1729, 2208])
growth_rate = (x1 - x0) / x0
#print(growth_rate)
doubling_time = 1.0 / np.log2(growth_rate + 1)

gamma = 1/14
beta = growth_rate + gamma

for val in beta:
    print(val)

0.3504013296578563
0.3066796197745145
0.32264098655951223
0.31417907386899635
0.28851301176587074
0.26174205908552883
0.21518330571075459


In [38]:
# la, nyc, slc county data from nytimes county level data
la = pd.read_csv("la_data.csv")
nyc = pd.read_csv("la_data.csv")
slc = pd.read_csv("la_data.csv")


In [43]:
# add admits to data for LA county.
la["admits"] = la.cases - la.cases.shift()
# Pretend customer gave us rows 60-65
la_train = la.iloc[60:66]
la_train = la_train[["date", "cases", "admits"]]
# Use rows 66-end to project
la_test = la.iloc[66:]
la_test = la_test[["date", "cases", "admits"]]

In [44]:
la_train

,date,cases,admits
60,2020-03-26,1216,417.0
61,2020-03-27,1465,249.0
62,2020-03-28,1804,339.0
63,2020-03-29,2136,332.0
64,2020-03-30,2474,338.0
65,2020-03-31,3011,537.0


In [46]:
gamma = 1/14

growth_rate = la_train.admits / la_train.cases
doubling_time = 1.0 / np.log2(growth_rate + 1)
beta = growth_rate + gamma

# Print doubling time and beta from each time step
for val1, val2 in zip(doubling_time, beta):
    print("doubling time: ", val1)
    print("beta: ", val2)
    print(" ")

doubling time:  2.3508306166837216
beta:  0.41435620300751874
 
doubling time:  4.415665203162323
beta:  0.24139444173573865
 
doubling time:  4.025238058736619
beta:  0.25934431422236304
 
doubling time:  4.797756627452325
beta:  0.2268592830390583
 
doubling time:  5.4126878376129435
beta:  0.208049428340455
 
doubling time:  4.223627535779114
beta:  0.2497746358589932
 


In [52]:
# This might do it for SIR from data?
population = 130000
market_share = 100
la_train["r"] = la_train.cases.shift()*gamma
la_train["s"] = population - la_train.cases
la_train["i"] = population - la_train["s"] - la_train["r"]
la_train[["date", "cases", "admits", "s", "i", "r"]]

,date,cases,admits,s,i,r
60,2020-03-26,1216,417.0,128784,NaN,NaN
61,2020-03-27,1465,249.0,128535,1378.142857,86.857143
62,2020-03-28,1804,339.0,128196,1699.357143,104.642857
63,2020-03-29,2136,332.0,127864,2007.142857,128.857143
64,2020-03-30,2474,338.0,127526,2321.428571,152.571429
65,2020-03-31,3011,537.0,126989,2834.285714,176.714286


In [47]:
# Generated Admits and Census Data (Pretend a client gave us these)
admits_df = pd.read_csv("admits_test.csv")
# census_df = pd.read_csv("census_test.csv")
sir_df = pd.read_csv("sir_test.csv")


In [56]:
# put cases on admits
admits_df["cases"] = np.empty(admits_df.shape[0])
admits_df["cases"] = 0
admits_df["total_float"] = admits_df.non_icu + admits_df.icu
admits_df["cases"] = np.cumsum(admits_df.total_float.shift(1))

sir_sample = sir_df.loc[sir_df.date >= "2020-05-10"]
sir_sample = sir_sample.loc[sir_sample.date < "2020-05-31"]

admits_sample = admits_df.loc[admits_df.date >= "2020-05-10"]
admits_sample = admits_sample.loc[admits_sample.date < "2020-05-31"]
admits_sample

,day,date,non_icu,icu,ventilators,total,cases,total_float
41,24,2020-05-10,265.280562,265.280562,2.652806,530.0,2810.642382,530.561124
42,25,2020-05-11,315.111446,315.111446,3.151114,630.0,3341.203506,630.222891
43,26,2020-05-12,374.221570,374.221570,3.742216,748.0,3971.426397,748.443140
44,27,2020-05-13,444.305473,444.305473,4.443055,888.0,4719.869537,888.610946
45,28,2020-05-14,527.353391,527.353391,5.273534,1054.0,5608.480483,1054.706783
46,29,2020-05-15,625.697165,625.697165,6.256972,1250.0,6663.187266,1251.394331
47,30,2020-05-16,742.060856,742.060856,7.420609,1484.0,7914.581597,1484.121712
48,31,2020-05-17,879.615442,879.615442,8.796154,1758.0,9398.703308,1759.230885
49,32,2020-05-18,1042.036305,1042.036305,10.420363,2084.0,11157.934193,2084.072610
50,33,2020-05-19,1233.561215,1233.561215,12.335612,2466.0,13242.006803,2467.122430


In [57]:
# Calculate SIR from those cases data
population = 1000000
market_share = 100
gamma = 1/14
hosp_rate = 0.0075 + 0.025

sir_calc = pd.DataFrame()
sir_calc["date"] = admits_sample.date
sir_calc["cases"] = admits_sample.cases
sir_calc["admits"] = admits_sample.total_float

sir_calc["i_calc"] = admits_sample.cases
sir_calc["s_calc"] = 0 # population - admits_sample.cases
sir_calc["r_calc"] = 0 # population - sir_calc["s_calc"] - sir_calc["r_calc"]

sir_calc["s"] = sir_sample.susceptible
sir_calc["i"] = sir_sample.infected
sir_calc["r"] = sir_sample.recovered

sir_calc["s_delta"] = sir_sample.susceptible - sir_calc.s_calc
sir_calc["i_delta"] = sir_sample.infected - sir_calc.i_calc
sir_calc["r_delta"] = sir_sample.recovered - sir_calc.r_calc

sir_calc[["date", "cases", "admits", "s", "i", "r", "s_calc", "i_calc", "r_calc", "s_delta", "i_delta", "r_delta"]]

,date,cases,admits,s,i,r,s_calc,i_calc,r_calc,s_delta,i_delta,r_delta
41,2020-05-10,2810.642382,530.561124,996656.796494,2426.128707,917.074799,0,2810.642382,0,996656.796494,-384.513675,917.074799
42,2020-05-11,3341.203506,630.222891,996026.573603,2883.056690,1090.369707,0,3341.203506,0,996026.573603,-458.146815,1090.369707
43,2020-05-12,3971.426397,748.443140,995278.130463,3425.567209,1296.302327,0,3971.426397,0,995278.130463,-545.859188,1296.302327
44,2020-05-13,4719.869537,888.610946,994389.519517,4069.494783,1540.985700,0,4719.869537,0,994389.519517,-650.374754,1540.985700
45,2020-05-14,5608.480483,1054.706783,993334.812734,4833.523367,1831.663898,0,5608.480483,0,993334.812734,-774.957116,1831.663898
46,2020-05-15,6663.187266,1251.394331,992083.418403,5739.666029,2176.915567,0,6663.187266,0,992083.418403,-923.521237,2176.915567
47,2020-05-16,7914.581597,1484.121712,990599.296692,6813.811596,2586.891712,0,7914.581597,0,990599.296692,-1100.770001,2586.891712
48,2020-05-17,9398.703308,1759.230885,988840.065807,8086.341652,3073.592541,0,9398.703308,0,988840.065807,-1312.361656,3073.592541
49,2020-05-18,11157.934193,2084.072610,986755.993197,9592.818430,3651.188373,0,11157.934193,0,986755.993197,-1565.115763,3651.188373
50,2020-05-19,13242.006803,2467.122430,984288.870767,11374.739544,4336.389689,0,13242.006803,0,984288.870767,-1867.267259,4336.389689


In [28]:
# Just scratchwork below.

# Doesn't change
population = 328200000
# Varies with time
for i0, r_act, cd in zip(x0, cum_rec, cum_dead):
    r_calc = i0 * 1/14
    s_calc = population - i0 - r_calc
    print("s: ", s_calc)
    print("i: ", i0)
    print("rc: ", r_calc)
    print("r_act: ", r_act+cd)
    print("  ")

s:  328162242.85714287
i:  35240
rc:  2517.1428571428573
r_act:  1140
  
s:  328151709.64285713
i:  45071
rc:  3219.3571428571427
r_act:  1697
  
s:  328140349.28571427
i:  55674
rc:  3976.714285714286
r_act:  2206
  
s:  328125364.28571427
i:  69660
rc:  4975.714285714285
r_act:  3172
  
s:  328107246.4285714
i:  86570
rc:  6183.571428571428
r_act:  4131
  
s:  328087111.0714286
i:  105363
rc:  7525.928571428572
r_act:  5341
  
s:  328065626.78571427
i:  125415
rc:  8958.214285714286
r_act:  6893
  


In [49]:
def sir(
    s: float, i: float, r: float, beta: float, gamma: float, n: float
) -> Tuple[float, float, float]:
    """The SIR model, one time step."""
    s_n = (-beta * s * i) + s
    i_n = (beta * s * i - gamma * i) + i
    r_n = gamma * i + r
    scale = n / (s_n + i_n + r_n)
    return s_n * scale, i_n * scale, r_n * scale


def sim_sir(
    s: float, i: float, r: float, gamma: float, i_day: int, beta: np.array
):
    """Simulate SIR model forward in time, returning a dictionary of daily arrays
    Parameter order has changed to allow multiple (beta, n_days)
    to reflect multiple changing social distancing policies.
    """
    s, i, r = (float(v) for v in (s, i, r))
    n = s + i + r
    d = i_day
    

    s_a = np.empty(n_days, "float")
    i_a = np.empty(n_days, "float")
    r_a = np.empty(n_days, "float")
    beta_a = np.empty(n_days, "float")

    index = 0
    for _ in range(n_days):
        beta = 
        d_a[index] = d
        s_a[index] = s
        i_a[index] = i
        r_a[index] = r
        index += 1

        s, i, r = sir(s, i, r, beta, gamma, n)
        d += 1

    d_a[index] = d
    s_a[index] = s
    i_a[index] = i
    r_a[index] = r
    return {
        "day": d_a,
        "susceptible": s_a,
        "infected": i_a,
        "recovered": r_a,
        "ever_infected": i_a + r_a
    }


def build_sim_sir_w_date_df(
    raw_df: pd.DataFrame,
    current_date: datetime,
    keys: Sequence[str],
) -> pd.DataFrame:
    day = raw_df.day
    return pd.DataFrame({
        "day": day,
        "date": day.astype('timedelta64[D]') + np.datetime64(current_date),
        **{
            key: raw_df[key]
            for key in keys
        }
    })

sim_sir()